In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import entropy


from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *

from tqdm import tqdm
tqdm.pandas()
import warnings


warnings.filterwarnings("ignore")

In [2]:
model_output=r"D:\Pranto\Working\LSA and Writeoff\LSA and Writeoff New Model\Writeoff\DATA\DATA\Model Output"

name1=r"Writeoff Model Jan24_Feb25"

# Import Train and Test data

In [3]:
path_main=r"D:\Pranto\Working\LSA and Writeoff\LSA and Writeoff New Model\Writeoff\DATA\DATA"

name=r"Writeoff model_jan24_Feb25_raw data with all variables V1_apply_binning V1"

In [4]:
try:
    df=pd.read_csv(rf"{path_main}\{name}.csv")
except:
    df=pd.read_excel(rf"{path_main}\{name}.xlsx")

print(df.shape)

df.head(3)

(1828624, 290)


,CONNO,Collectable,Collection,RR,WOFF,MONTH_YEAR,Payment_YN,RR_YN,MONTH_YEAR_V1,TENURE,...,PRODUCT_WoE,FINPROD_WoE,APPLICATION_USAGE_WoE,ASSET_CATEGORY_WoE,ASSET_CATEGORY2_WoE,BRANCH_WoE,CUSTOMER_SEGMENT_WoE,CUST_SEG_WoE,DSA_DLR_WoE,PRODUCT_New_WoE
0,29372,251550.0,0.0,RF,Y,23376,0,0,JAN24,45,...,1.760924,1.469896,-0.132104,0.369879,0.32466,0.176409,0.163029,0.117918,1.507485,1.760905
1,29372,251550.0,0.0,RF,Y,23407,0,0,FEB24,45,...,1.760924,1.469896,-0.132104,0.369879,0.32466,0.176409,0.163029,0.117918,1.507485,1.760905
2,29372,251550.0,0.0,RF,Y,23436,0,0,MAR24,45,...,1.760924,1.469896,-0.132104,0.369879,0.32466,0.176409,0.163029,0.117918,1.507485,1.760905


In [5]:
df['MONTH_YEAR_V1'].unique()

array(['JAN24', 'FEB24', 'MAR24', 'APR24', 'MAY24', 'JUN24', 'JUL24',
       'AUG24', 'SEP24', 'OCT24', 'NOV24', 'DEC24', 'JAN25', 'FEB25'],
      dtype=object)

# Primary Key

In [6]:
# df['Primary_Key']=df['CONNO'].astype(str)+"_"+df['MONTH'].astype(str)

# df.columns

In [7]:
# df['MONTH'].unique()

In [8]:
# df['Primary_Key'].unique()

In [9]:
# # Export

# df.to_csv(rf"{path_main}\{name}_with Primary Key.csv",index=False)

In [ ]:
train_month=['JAN24', 'FEB24', 'MAR24', 'APR24', 'MAY24', 'JUN24', 'JUL24',
       'AUG24', 'SEP24', 'OCT24', 'NOV24', 'DEC24']
 
test_month=['JAN25','FEB25']
 
filter_column='MONTH_YEAR_V1'
 
target_variable='TARGET_VARIABLE'

columns_last_name='WoE'

In [11]:
# train_month=['JUL23','AUG23','SEP23','OCT23','NOV23','DEC23','JAN24','FEB24','MAR24','APR24','MAY24','JUN24']

# test_month=['JUL24','AUG24','SEP24']

# filter_column='MONTH'

# target_variable='TARGET_VARIABLE_V1'

In [ ]:
# Train Data

train_data=df[df[filter_column].isin(train_month)]


X_train = train_data.filter(like=columns_last_name)


print("Shape of train Data====",train_data.shape,'\n',"Shape of X_train===",X_train.shape,'\n',"Unique month===",train_data[filter_column].unique())


print("===================================================")

# Create test data dictionary for CSI table
test_datasets = {}
# Test Data
for i in test_month:
    globals()[f'test_data_{i}']=df[df[filter_column].eq(i)]

    globals()[f'X_test_{i}'] = globals()[f'test_data_{i}'].filter(like=columns_last_name)

    print(f"Shape of test Data {i}====",globals()[f'test_data_{i}'].shape,'\n',f"Shape of X_test_{i}",globals()[f'X_test_{i}'].shape,'\n',"Unique month===",globals()[f'test_data_{i}'][filter_column].unique())


    


    # Add each test dataset to the dictionary
    test_datasets[f'Test_{i}'] = globals()[f'X_test_{i}']






Shape of train Data==== (1554348, 290) 
 Shape of X_train=== (1554348, 56) 
 Unique month=== ['JAN24' 'FEB24' 'MAR24' 'APR24' 'MAY24' 'JUN24' 'JUL24' 'AUG24' 'SEP24'
 'OCT24' 'NOV24' 'DEC24']
Shape of test Data JAN25==== (137391, 290) 
 Shape of X_test_JAN25 (137391, 56) 
 Unique month=== ['JAN25']
Shape of test Data FEB25==== (136885, 290) 
 Shape of X_test_FEB25 (136885, 56) 
 Unique month=== ['FEB25']


In [13]:
test_datasets

{'Test_JAN25':          Collectable_WoE  IRR_CUSTOMER_WoE  ASSET_COST_WoE  \
 12              0.193237          0.011975        0.105061   
 26              0.193237          0.011975        0.105061   
 40              0.193237          0.011975        0.105061   
 54              0.193237          0.011975        0.105061   
 68              0.193237          0.011975        0.105061   
 ...                  ...               ...             ...   
 1828491         0.193237          0.011975        0.105061   
 1828514         0.193237          0.011975        0.105061   
 1828545         0.193237          0.011975        0.105061   
 1828569         0.193237          0.011975        0.105061   
 1828579         0.193237          0.011975        0.105061   
 
          COLLECTION_TILL_DATE_WoE  ODAMT_WoE  ODBKT_WoE   MOB_WoE  SOHPOD_WoE  \
 12                       0.131316   0.990000   1.840959  1.734840    0.898436   
 26                       0.131316   0.990000   1.840959  1.7348

# CSI and VIF table

In [14]:
def calculate_csi_by_value(train, test):
    """
    Calculate CSI for each unique value for each variable comparing train and test dataset.
    Returns a DataFrame with value-level details.
    """
    csi_details = []

    for column in train.columns:
        # Calculate count % for each unique value in train and test datasets
        train_value_counts = train[column].value_counts(normalize=True)
        test_value_counts = test[column].value_counts(normalize=True)

        # Align indices so both series have all unique values across train and test
        all_values = train_value_counts.index.union(test_value_counts.index)
        train_value_counts = train_value_counts.reindex(all_values).fillna(1e-10)
        test_value_counts = test_value_counts.reindex(all_values).fillna(1e-10)

        # Calculate CSI for each unique value
        value_csi = (train_value_counts - test_value_counts) * np.log(train_value_counts / test_value_counts)

        # Append data for each unique value
        for value in all_values:
            csi_details.append({
                'Variable': column,
                'Value': value,
                'Train %': train_value_counts[value],
                'Test %': test_value_counts[value],
                'Value CSI': value_csi[value]
            })

    return pd.DataFrame(csi_details)

def calculate_total_csi_and_decision(csi_df, threshold_stable=0.1, threshold_watch=0.25):
    """
    Calculate total CSI and decision based on total CSI for each variable.
    """
    total_csi_df = csi_df.groupby('Variable')['Value CSI'].sum().reset_index(name='Total CSI')

    # Determine stability decision
    total_csi_df['Decision'] = np.where(
        total_csi_df['Total CSI'] < threshold_stable, 'Stable',
        np.where(total_csi_df['Total CSI'] <= threshold_watch, 'On Watch', 'Unstable')
    )

    return total_csi_df

def generate_detailed_csi_table(train, test_datasets):
    """
    Generate a detailed CSI table with Total CSI and Decision columns for train and each test dataset.
    """
    detailed_csi_df = pd.DataFrame()
    # pass

    # Detailed CSI for train
    train_csi_details = calculate_csi_by_value(train, train)
    train_total_csi = calculate_total_csi_and_decision(train_csi_details)
    train_csi_details = train_csi_details.merge(train_total_csi, on='Variable')
    train_csi_details = train_csi_details.rename(columns={
        'Train %': 'Train %',
        'Total CSI': 'Train Total CSI',
        'Decision': 'Train Decision'
    })

    # Start with the train data in detailed format
    # detailed_csi_df = train_csi_details[['Variable', 'Value', 'Train %', 'Value CSI', 'Train Total CSI', 'Train Decision']]
    detailed_csi_df = train_csi_details[['Variable', 'Value', 'Train %']]
    
    # Detailed CSI for each test dataset
    for test_name, test_data in test_datasets.items():
        # Calculate CSI for each test dataset
        test_csi_details = calculate_csi_by_value(train, test_data)
        test_total_csi = calculate_total_csi_and_decision(test_csi_details)

        # Rename columns for the specific test dataset
        test_csi_details = test_csi_details.rename(columns={
            'Test %': f'{test_name} %',
            'Value CSI': f'{test_name} Value CSI'
        })
        test_total_csi = test_total_csi.rename(columns={
            'Total CSI': f'{test_name} Total CSI',
            'Decision': f'{test_name} Decision'
        })

        # Merge test bin details with total CSI and decision
        test_csi_details = test_csi_details.merge(test_total_csi, on='Variable')

        # Merge test details into the main detailed CSI DataFrame
        detailed_csi_df = detailed_csi_df.merge(test_csi_details[['Variable', 'Value', f'{test_name} %', f'{test_name} Value CSI', f'{test_name} Total CSI', f'{test_name} Decision']],
                                                on=['Variable', 'Value'],
                                                how='left')

    return detailed_csi_df





# Detailed CSI

In [15]:
test_datasets

{'Test_JAN25':          Collectable_WoE  IRR_CUSTOMER_WoE  ASSET_COST_WoE  \
 12              0.193237          0.011975        0.105061   
 26              0.193237          0.011975        0.105061   
 40              0.193237          0.011975        0.105061   
 54              0.193237          0.011975        0.105061   
 68              0.193237          0.011975        0.105061   
 ...                  ...               ...             ...   
 1828491         0.193237          0.011975        0.105061   
 1828514         0.193237          0.011975        0.105061   
 1828545         0.193237          0.011975        0.105061   
 1828569         0.193237          0.011975        0.105061   
 1828579         0.193237          0.011975        0.105061   
 
          COLLECTION_TILL_DATE_WoE  ODAMT_WoE  ODBKT_WoE   MOB_WoE  SOHPOD_WoE  \
 12                       0.131316   0.990000   1.840959  1.734840    0.898436   
 26                       0.131316   0.990000   1.840959  1.7348

In [16]:

# Generate the detailed CSI table
detailed_csi_table = generate_detailed_csi_table(X_train, test_datasets)

# Display the detailed CSI table
pd.set_option('display.max_columns', None)  # To show all columns
print("Detailed CSI Table:")
detailed_csi_table

Detailed CSI Table:


,Variable,Value,Train %,Test_JAN25 %,Test_JAN25 Value CSI,Test_JAN25 Total CSI,Test_JAN25 Decision,Test_FEB25 %,Test_FEB25 Value CSI,Test_FEB25 Total CSI,Test_FEB25 Decision
0,Collectable_WoE,0.193237,0.899909,0.907905,7.074513e-05,0.000737,Stable,0.908091,7.405883e-05,0.000772,Stable
1,Collectable_WoE,-0.960909,0.100091,0.092095,6.658490e-04,0.000737,Stable,0.091909,6.978022e-04,0.000772,Stable
2,IRR_CUSTOMER_WoE,0.011975,0.995808,0.995873,4.269260e-09,0.000001,Stable,0.996062,6.503438e-08,0.000016,Stable
3,IRR_CUSTOMER_WoE,-1.366956,0.004192,0.004127,1.022137e-06,0.000001,Stable,0.003938,1.593934e-05,0.000016,Stable
4,ASSET_COST_WoE,0.105061,0.940659,0.934807,3.652275e-05,0.000587,Stable,0.935011,3.400945e-05,0.000547,Stable
...,...,...,...,...,...,...,...,...,...,...,...
118,DSA_DLR_WoE,1.507485,0.328549,0.315115,5.608532e-04,0.002307,Stable,0.315484,5.301951e-04,0.002101,Stable
119,DSA_DLR_WoE,0.000000,0.214326,0.203842,5.258339e-04,0.002307,Stable,0.204544,4.569597e-04,0.002101,Stable
120,PRODUCT_New_WoE,1.760905,0.671471,0.652102,5.669190e-04,0.001844,Stable,0.653687,4.773532e-04,0.001630,Stable
121,PRODUCT_New_WoE,-0.761302,0.269190,0.288010,1.271825e-03,0.001844,Stable,0.287088,1.152075e-03,0.001630,Stable


# Variablewise Final decision of CSI

In [17]:
# Assuming detailed_csi_df is your DataFrame
columns_with_decision = [col for col in detailed_csi_table.columns if 'decision' in col.lower()]

detailed_csi_table1=detailed_csi_table[['Variable']+columns_with_decision]

monthly_decision_table=detailed_csi_table1.drop_duplicates()

monthly_decision_table.head(3)

,Variable,Test_JAN25 Decision,Test_FEB25 Decision
0,Collectable_WoE,Stable,Stable
2,IRR_CUSTOMER_WoE,Stable,Stable
4,ASSET_COST_WoE,Stable,Stable


# Select those variables who are either stable or onwatch

In [18]:
# Filter for variables that are either 'Stable' or 'On Watch' for all test months
stable_or_onwatch_vars = monthly_decision_table[
    monthly_decision_table.loc[:, monthly_decision_table.columns.str.endswith('Decision')].apply(
        lambda row: all(decision in ['Stable', 'On Watch'] for decision in row), axis=1)
]['Variable'].tolist()

print("Variables stable or on watch across all test months:")
stable_or_onwatch_vars


Variables stable or on watch across all test months:


['Collectable_WoE',
 'IRR_CUSTOMER_WoE',
 'ASSET_COST_WoE',
 'COLLECTION_TILL_DATE_WoE',
 'ODAMT_WoE',
 'ODBKT_WoE',
 'MOB_WoE',
 'SOHPOD_WoE',
 'max_DPD_WoE',
 'max_odbkt_WoE',
 'last_Month_coll_WoE',
 'last_3Month_coll_WoE',
 'last_6Month_coll_WoE',
 'MaxBkt_3M_WoE',
 'MaxBkt_6M_WoE',
 'MaxBkt_9M_WoE',
 'MaxBkt_12M_WoE',
 'MaxBkt_24M_WoE',
 'CountInBkt_1_24M_WoE',
 'CountInBkt_2_24M_WoE',
 'CountInBkt_3_24M_WoE',
 'CountInBkt_1Plus_3M_WoE',
 'CountInBkt_2Plus_3M_WoE',
 'CountInBkt_3Plus_3M_WoE',
 'CountInBkt_3Plus_6M_WoE',
 'EverInBkt_1Plus_6M_WoE',
 'EverInBkt_1Plus_9M_WoE',
 'EverInBkt_1Plus_12M_WoE',
 'EverInBkt_2Plus_3M_WoE',
 'EverInBkt_2Plus_6M_WoE',
 'EverInBkt_3Plus_3M_WoE',
 'EverInBkt_3Plus_6M_WoE',
 'EverInBkt_3Plus_9M_WoE',
 'EverInBkt_3Plus_12M_WoE',
 'Last_1M_del_WoE',
 'Last_2M_del_WoE',
 'RollFw1_WoE',
 'RollFw2_WoE',
 'RollFw3_WoE',
 'RollFw6_WoE',
 'Roll_stable_1_WoE',
 'Roll_stable_2_WoE',
 'Roll_stable_3_WoE',
 'Roll_stable_6_WoE',
 'remain_per_WoE',
 'SCHEME_FINP

# Filter the Variables

In [19]:


# Step 1: Calculating VIF
def calculate_vif(df):
    vif_df = pd.DataFrame()
    vif_df['Variable'] = df.columns
    vif_df['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_df

# Assuming X_train and stable_or_onwatch_vars are defined
X_train_V1 = X_train[stable_or_onwatch_vars]

# Step 2: Calculate VIF for X_train_V1
vif_df = calculate_vif(X_train_V1)
X_train_filtered = X_train_V1.loc[:, vif_df[vif_df['VIF'] < 5]['Variable']]

print("Done VIF calculation")

# Step 3: Filter columns for each test set in test_month list
# test_month = ['JUL24', 'AUG24', 'SEP24']  # Add any other months as needed
filtered_test_datasets = {}

for month in tqdm(test_month,desc="Processing Time"):
    # Access the test dataset dynamically
    X_test = globals()[f'X_test_{month}'][stable_or_onwatch_vars]
    filtered_test_datasets[f'X_test_{month}_filtered'] = X_test[X_train_filtered.columns]

# Print results for verification
print("Filtered Training Columns:======================", X_train_filtered.columns)




Done VIF calculation


Processing Time: 100%|██████████| 2/2 [00:00<00:00, 13.69it/s]

Filtered Training Columns:====================== Index(['IRR_CUSTOMER_WoE', 'ASSET_COST_WoE', 'COLLECTION_TILL_DATE_WoE',
       'SOHPOD_WoE', 'remain_per_WoE', 'APPLICATION_USAGE_WoE', 'BRANCH_WoE',
       'CUSTOMER_SEGMENT_WoE', 'CUST_SEG_WoE', 'DSA_DLR_WoE'],
      dtype='object')


In [20]:
# Merge Vif With Monthly CSI table

monthly_decision_table1=monthly_decision_table.merge(vif_df,on='Variable',how='left')

# Add the filter column based on VIF and Monthly Decision criteria
monthly_decision_table1['Variable Importance'] = np.where(
    (monthly_decision_table1['VIF'] < 5) & 
    monthly_decision_table1.loc[:, monthly_decision_table1.columns.str.endswith('Decision')].progress_apply(
        lambda row: all(decision in ['Stable', 'On Watch'] for decision in row), axis=1
    ),
    'Yes', 
    'No'
)

print(monthly_decision_table1['Variable Importance'].value_counts())
monthly_decision_table1.head(3)

100%|██████████| 56/56 [00:00<00:00, 18732.04it/s]

Variable Importance
No     46
Yes    10
Name: count, dtype: int64


,Variable,Test_JAN25 Decision,Test_FEB25 Decision,VIF,Variable Importance
0,Collectable_WoE,Stable,Stable,5.636485,No
1,IRR_CUSTOMER_WoE,Stable,Stable,1.017158,Yes
2,ASSET_COST_WoE,Stable,Stable,1.290536,Yes


# Export

In [21]:
with pd.ExcelWriter(rf"{model_output}\{name1}_Detailed_CSI V1.xlsx",engine='xlsxwriter') as want:
    monthly_decision_table1.to_excel(want,sheet_name="Variable Wise CSI",index=False)
    detailed_csi_table.to_excel(want,sheet_name="Detailed CSI",index=False)
    vif_df.to_excel(want,sheet_name="VIF",index=False)

In [22]:
detailed_csi_table

,Variable,Value,Train %,Test_JAN25 %,Test_JAN25 Value CSI,Test_JAN25 Total CSI,Test_JAN25 Decision,Test_FEB25 %,Test_FEB25 Value CSI,Test_FEB25 Total CSI,Test_FEB25 Decision
0,Collectable_WoE,0.193237,0.899909,0.907905,7.074513e-05,0.000737,Stable,0.908091,7.405883e-05,0.000772,Stable
1,Collectable_WoE,-0.960909,0.100091,0.092095,6.658490e-04,0.000737,Stable,0.091909,6.978022e-04,0.000772,Stable
2,IRR_CUSTOMER_WoE,0.011975,0.995808,0.995873,4.269260e-09,0.000001,Stable,0.996062,6.503438e-08,0.000016,Stable
3,IRR_CUSTOMER_WoE,-1.366956,0.004192,0.004127,1.022137e-06,0.000001,Stable,0.003938,1.593934e-05,0.000016,Stable
4,ASSET_COST_WoE,0.105061,0.940659,0.934807,3.652275e-05,0.000587,Stable,0.935011,3.400945e-05,0.000547,Stable
...,...,...,...,...,...,...,...,...,...,...,...
118,DSA_DLR_WoE,1.507485,0.328549,0.315115,5.608532e-04,0.002307,Stable,0.315484,5.301951e-04,0.002101,Stable
119,DSA_DLR_WoE,0.000000,0.214326,0.203842,5.258339e-04,0.002307,Stable,0.204544,4.569597e-04,0.002101,Stable
120,PRODUCT_New_WoE,1.760905,0.671471,0.652102,5.669190e-04,0.001844,Stable,0.653687,4.773532e-04,0.001630,Stable
121,PRODUCT_New_WoE,-0.761302,0.269190,0.288010,1.271825e-03,0.001844,Stable,0.287088,1.152075e-03,0.001630,Stable
